In [91]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import os
import datetime

In [92]:
#Load Matches data
partidos=pd.read_excel("Eurocupsf.xlsx")
#Separate teams in different columns
partidos["equipo 1"]=partidos['Team 1'].astype(str)
partidos["equipo 2"]=partidos['Team 2'].astype(str)

In [93]:
#Cleaning countries names
partidos['equipo 1'].replace({'Israel -':'Israel',
                              'Andorra -':'Andorra',
                              'Israel - Belgium':'Israel',
                              'Cyprus -':'Cyprus',
                              'Netherlands -':'Netherlands',
                              'Croatia -':'Croatia',
                              'Slovakia -':'Slovakia',
                              'Austria -':'Austria',
                              'North Macedonia -':'Macedonia',
                              'North Macedonia':'Macedonia',
                              'Kazakhstan -':'Kazakhstan',
                              'Belgium -':'Belgium',
                              'England -':'England',
                              'Luxembourg -':'Luxembourg',
                              'Portugal -':'Portugal',
                              'Albania -':'Albania',
                              'Moldova -':'Moldova',
                              'Wales -':'Wales',
                              'Georgia -':'Georgia',
                              'Gibraltar -':'Gibraltar',
                              'Sweden -':'Sweden',
                              'Italy -':'Italy',
                              'Hungary -':'Hungary',
                              'Poland -':'Poland',
                              'Slovenia -':'Slovenia',
                              'San Marino -':'San Marino',
                              'Kosovo -':'Kosovo',
                              'Montenegro -':'Montenegro',
                              'Turkey -':'Turkey',
                              'France -':'France',
                              'Ireland -':'Ireland',
                              'Switzerland -':'Switzerland',
                              'Norway -':'Norway',
                              'Romania -':'Romania',
                              'Armenia -':'Armenia',
                              'Estonia -':'Estonia',
                              'Belarus -':'Belarus',
                              'Azerbaijan -':'Azerbaijan',
                              'Iceland -':'Iceland',
                              'Scotland -':'Scotland',
                              'Finland -':'Finland',
                              'Bulgaria -':'Bulgaria',
                              'Serbia -':'Serbia',
                              'Ukraine -':'Ukraine',
                              'Denmark -':'Denmark',
                              'Malta -':'Malta',
                              'Latvia -':'Latvia',
                              'Germany -':'Germany',
                              'Russia -':'Russia',
                              'Greece -':'Greece',
                              'Lithuania -':'Lithuania',
                              'Spain -':'Spain',
                              'Bosnia-Herzegovina -':'Bosnia-Herzegovina',
                              'Czech':'Czech Republic',
                              'Northern':'Northern Ireland',
                              'Malta - Faroe':'Malta',
                              'Russia - San':'Russia',
                              'Israel - North':'Israel',
                              'Kosovo - Czech':'Kosovo',
                              'Spain - Faroe':'Spain',
                              'Belgium - San':'Belgium',
                              'Poland - North':'Poland',
                              'Scotland - San':'Scotland',
                              'Norway - Faroe':'Norway',
                              'Austria - North':'Austria',
                              'Sweden - Faroe':'Sweden',
                              'Georgia - North':'Georgia',
                              'North':'Macedonia',}, inplace=True)
for i in partidos['equipo 2']:
    i.replace('_x000D_','')
partidos['equipo 2'].replace({'BosniaHerzegovina':'Bosnia-Herzegovina',
                              'Republic  Netherlands':'Netherlands',
                              '01':'Belgium',
                              'Ireland  Faroe Islands':'Faroe Islands',
                              'Ireland  Finland':'Finland',
                              'Ireland  Romania':'Romania',
                              'Ireland  Hungary':'Hungary',
                              'Ireland  Greece':'Greece',
                              'Ireland  Estonia':'Estonia',
                              'Islands':'Faroe Islands',
                              'Ireland  Belarus':'Belarus',
                              'North Macedonia':'Macedonia',
                              'Marino':'San Marino',
                              'Macedonia  Austria':'Austria',
                              'Republic':'Czech Republic',
                              'Ireland  Germany':'Germany',
                              'Macedonia  Slovenia':'Slovenia',
                              'Macedonia  Israel':'Israel',
                              'Macedonia  Kosovo':'Kosovo',
                              'Ireland  Slovakia':'Slovakia',
                              'Ireland  Netherlands':'Netherlands'}, inplace=True)

In [94]:
#Drop empty fields
partidos
partidos.dropna()

,Date,Stage,Team 1,Team 2,Final result,Goals,equipo 1,equipo 2
0,16-08-2006,QUALIFYING STAGE – GROUP A,Belgium,Kazakhstan,0-0 (0-0),-,Belgium,Kazakhstan
1,2-09-2006,QUALIFYING STAGE – GROUP A,Serbia,Azerbaijan,1-0 (0-0),1-0 (Žigić 72’)[Incidents Game played behind c...,Serbia,Azerbaijan
2,2-09-2006,QUALIFYING STAGE – GROUP A,Poland,Finland,1-3 (0-0),0-1 (Litmanen 54’); 0-2 (Litmanen [p.] 76’); ...,Poland,Finland
3,6-09-2006,QUALIFYING STAGE – GROUP A,Azerbaijan,Kazakhstan,1-1 (1-1),1-0 (Ladaga 16’); 1-1 (Bjakov 36’),Azerbaijan,Kazakhstan
4,6-09-2006,QUALIFYING STAGE – GROUP A,Finland,Portugal,1-1 (1-1),1-0 (Johansson 22’); 1-1 (Nuno Gomes 42’),Finland,Portugal
...,...,...,...,...,...,...,...,...
1210,28-06-2021,1/8 FINAL,France,Switzerland,3-3,GOALS 0-1 (Seferović 15’); 1-1 (Benzema 57’); ...,France,Switzerland
1211,29-06-2021,1/8 FINAL,England,Germany,2-0,GOALS 1-0 (Sterling 75’); 2-0 (Kane 86’),England,Germany
1212,6-07-2021,1/2 FINAL,Italy,Spain,1-1,GOALS 1-0 (Chiesa 60’); 1-1 (Morata 80’),Italy,Spain
1213,7-07-2021,1/2 FINAL,England,Denmark,2-1,GOALS 0-1 (Damsgaard 30’); 1-1 (Kjær [o.g.] 39...,England,Denmark


In [96]:
#Import dictionary with capitals
import json
with open('capitales.json', 'r') as JSON:
       json_dict = json.load(JSON)

In [95]:
#Import dictionary with acronyms
acronyms = pd.read_csv('Acronyms.csv')
acronyms_dict = {}
for index,country in enumerate(acronyms['Country']):
    acronyms_dict[country] = acronyms['Acronym'][index]

In [97]:
capitales={}
for i in range(len(json_dict)):
    capitales[json_dict[i]["country"]]=json_dict[i]["city"]


In [98]:
capitales.update({'Luxembourg': 'Luxembourg'})
capitales.values()

dict_values(['Kabul', 'Tirana', 'Alger', 'Fagatogo', 'Andorra la Vella', 'Luanda', 'The Valley', None, "Saint John's", 'Buenos Aires', 'Yerevan', 'Oranjestad', 'Canberra', 'Wien', 'Baku', 'Nassau', 'al-Manama', 'Dhaka', 'Bridgetown', 'Minsk', 'Bruxelles [Brussel]', 'Belmopan', 'Porto-Novo', 'Hamilton', 'Thimphu', 'La Paz', 'Sarajevo', 'Gaborone', None, 'BrasÃ\xadlia', None, 'Bandar Seri Begawan', 'Sofia', 'Ouagadougou', 'Bujumbura', 'Phnom Penh', 'Yaound', 'Ottawa', 'Praia', 'George Town', 'Bangui', "N'Djam", 'Santiago de Chile', 'Peking', 'Flying Fish Cove', 'West Island', 'Santaf', 'Moroni', 'Brazzaville', 'Avarua', 'San JosÃ©', 'Zagreb', 'La Habana', 'Nicosia', 'Praha', 'Copenhagen', 'Djibouti', 'Roseau', 'Santo Domingo de Guzm', 'Dili', 'Quito', 'Cairo', 'San Salvador', 'London', 'Malabo', 'Asmara', 'Tallinn', 'Addis Abeba', 'Stanley', 'TÃ³rshavn', 'Suva', 'Helsinki [Helsingfors]', 'Paris', 'Cayenne', 'Papeete', None, 'Libreville', 'Banjul', 'Tbilisi', 'Berlin', 'Accra', 'Gibraltar

In [99]:
#Assign capitals and acronyms to countries
for i in range(len(partidos["equipo 1"])):
    partidos["capitales1"] = partidos["equipo 1"].apply(lambda x: capitales.get(x))
    partidos["acronimo1"] = partidos["equipo 1"].apply(lambda x: acronyms_dict.get(x))
for i in range(len(partidos["equipo 2"])):
    partidos["capitales2"] = partidos["equipo 2"].apply(lambda x: capitales.get(x))
    partidos["acronimo2"] = partidos["equipo 2"].apply(lambda x: acronyms_dict.get(x))

In [100]:
partidos.dropna()

,Date,Stage,Team 1,Team 2,Final result,Goals,equipo 1,equipo 2,capitales1,acronimo1,capitales2,acronimo2
0,16-08-2006,QUALIFYING STAGE – GROUP A,Belgium,Kazakhstan,0-0 (0-0),-,Belgium,Kazakhstan,Bruxelles [Brussel],BE,Astana,KZ
1,2-09-2006,QUALIFYING STAGE – GROUP A,Serbia,Azerbaijan,1-0 (0-0),1-0 (Žigić 72’)[Incidents Game played behind c...,Serbia,Azerbaijan,Belgrade,RS,Baku,AZ
2,2-09-2006,QUALIFYING STAGE – GROUP A,Poland,Finland,1-3 (0-0),0-1 (Litmanen 54’); 0-2 (Litmanen [p.] 76’); ...,Poland,Finland,Warszawa,PL,Helsinki [Helsingfors],FI
3,6-09-2006,QUALIFYING STAGE – GROUP A,Azerbaijan,Kazakhstan,1-1 (1-1),1-0 (Ladaga 16’); 1-1 (Bjakov 36’),Azerbaijan,Kazakhstan,Baku,AZ,Astana,KZ
4,6-09-2006,QUALIFYING STAGE – GROUP A,Finland,Portugal,1-1 (1-1),1-0 (Johansson 22’); 1-1 (Nuno Gomes 42’),Finland,Portugal,Helsinki [Helsingfors],FI,Lisboa,PT
...,...,...,...,...,...,...,...,...,...,...,...,...
1210,28-06-2021,1/8 FINAL,France,Switzerland,3-3,GOALS 0-1 (Seferović 15’); 1-1 (Benzema 57’); ...,France,Switzerland,Paris,FR,Bern,CH
1211,29-06-2021,1/8 FINAL,England,Germany,2-0,GOALS 1-0 (Sterling 75’); 2-0 (Kane 86’),England,Germany,London,ENG,Berlin,DE
1212,6-07-2021,1/2 FINAL,Italy,Spain,1-1,GOALS 1-0 (Chiesa 60’); 1-1 (Morata 80’),Italy,Spain,Roma,IT,Madrid,ES
1213,7-07-2021,1/2 FINAL,England,Denmark,2-1,GOALS 0-1 (Damsgaard 30’); 1-1 (Kjær [o.g.] 39...,England,Denmark,London,ENG,Copenhagen,DK


In [101]:
import datetime
#Transform date into format for twitter
for i in range(len(partidos["Date"])):
    partidos["Date"][i]=datetime.datetime.strptime(partidos["Date"][i].replace('_x000D_',''), "%d-%m-%Y").strftime("%Y-%m-%d")

In [105]:
#Create keywords for search
partidos['kw1'] = partidos[['equipo 1', 'equipo 2']].agg(''.join, axis=1)
partidos['kw2'] = partidos[['equipo 1', 'equipo 2']].agg('-'.join, axis=1)
partidos['kw3'] = partidos[['equipo 1', 'equipo 2']].agg('_'.join, axis=1)
partidos['kw4'] = partidos[['equipo 1', 'equipo 2']].agg('vs'.join, axis=1)
partidos['kw5'] = partidos[['acronimo1', 'acronimo2']].agg(''.join, axis=1)
partidos['kw6'] = partidos[['acronimo1', 'acronimo2']].agg('-'.join, axis=1)
partidos['kw7'] = partidos[['acronimo1', 'acronimo2']].agg('_'.join, axis=1)
partidos['kw8'] = partidos[['acronimo1', 'acronimo2']].agg('vs'.join, axis=1)
partidos['kw9'] = partidos[['equipo 2', 'equipo 1']].agg(''.join, axis=1)
partidos['kw10'] = partidos[['equipo 2', 'equipo 1']].agg('-'.join, axis=1)
partidos['kw11'] = partidos[['equipo 2', 'equipo 1']].agg('_'.join, axis=1)
partidos['kw12'] = partidos[['equipo 2', 'equipo 1']].agg('vs'.join, axis=1)
partidos['kw13'] = partidos[['acronimo2', 'acronimo1']].agg(''.join, axis=1)
partidos['kw14'] = partidos[['acronimo2', 'acronimo1']].agg('-'.join, axis=1)
partidos['kw15'] = partidos[['acronimo2', 'acronimo1']].agg('_'.join, axis=1)
partidos['kw16'] = partidos[['acronimo2', 'acronimo1']].agg('vs'.join, axis=1)

In [106]:
#Function to search tweets using keywords, date and place.
def scrape_tweets(keyword,since,city):
    date_1 = datetime.datetime.strptime(since, '%Y-%m-%d')
    start_date = date_1 - datetime.timedelta(days=2)
    start = start_date.strftime('%Y-%m-%d')
    end_date = date_1 + datetime.timedelta(days=2)
    final = end_date.strftime('%Y-%m-%d')
    text=f"{keyword} since:{start} until:{final} "
    data = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    text).get_items(), 5000))
    try:
        data.to_csv(f"{since}+{city}.csv")
    except:
        data.to_csv(f"{since}.csv")
        
    return data

In [107]:
partidos

,Date,Stage,Team 1,Team 2,Final result,Goals,equipo 1,equipo 2,capitales1,acronimo1,...,kw7,kw8,kw9,kw10,kw11,kw12,kw13,kw14,kw15,kw16
0,2006-08-16,QUALIFYING STAGE – GROUP A,Belgium,Kazakhstan,0-0 (0-0),-,Belgium,Kazakhstan,Bruxelles [Brussel],BE,...,BE_KZ,BEvsKZ,KazakhstanBelgium,Kazakhstan-Belgium,Kazakhstan_Belgium,KazakhstanvsBelgium,KZBE,KZ-BE,KZ_BE,KZvsBE
1,2006-09-02,QUALIFYING STAGE – GROUP A,Serbia,Azerbaijan,1-0 (0-0),1-0 (Žigić 72’)[Incidents Game played behind c...,Serbia,Azerbaijan,Belgrade,RS,...,RS_AZ,RSvsAZ,AzerbaijanSerbia,Azerbaijan-Serbia,Azerbaijan_Serbia,AzerbaijanvsSerbia,AZRS,AZ-RS,AZ_RS,AZvsRS
2,2006-09-02,QUALIFYING STAGE – GROUP A,Poland,Finland,1-3 (0-0),0-1 (Litmanen 54’); 0-2 (Litmanen [p.] 76’); ...,Poland,Finland,Warszawa,PL,...,PL_FI,PLvsFI,FinlandPoland,Finland-Poland,Finland_Poland,FinlandvsPoland,FIPL,FI-PL,FI_PL,FIvsPL
3,2006-09-06,QUALIFYING STAGE – GROUP A,Azerbaijan,Kazakhstan,1-1 (1-1),1-0 (Ladaga 16’); 1-1 (Bjakov 36’),Azerbaijan,Kazakhstan,Baku,AZ,...,AZ_KZ,AZvsKZ,KazakhstanAzerbaijan,Kazakhstan-Azerbaijan,Kazakhstan_Azerbaijan,KazakhstanvsAzerbaijan,KZAZ,KZ-AZ,KZ_AZ,KZvsAZ
4,2006-09-06,QUALIFYING STAGE – GROUP A,Finland,Portugal,1-1 (1-1),1-0 (Johansson 22’); 1-1 (Nuno Gomes 42’),Finland,Portugal,Helsinki [Helsingfors],FI,...,FI_PT,FIvsPT,PortugalFinland,Portugal-Finland,Portugal_Finland,PortugalvsFinland,PTFI,PT-FI,PT_FI,PTvsFI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,2021-06-28,1/8 FINAL,France,Switzerland,3-3,GOALS 0-1 (Seferović 15’); 1-1 (Benzema 57’); ...,France,Switzerland,Paris,FR,...,FR_CH,FRvsCH,SwitzerlandFrance,Switzerland-France,Switzerland_France,SwitzerlandvsFrance,CHFR,CH-FR,CH_FR,CHvsFR
1211,2021-06-29,1/8 FINAL,England,Germany,2-0,GOALS 1-0 (Sterling 75’); 2-0 (Kane 86’),England,Germany,London,ENG,...,ENG_DE,ENGvsDE,GermanyEngland,Germany-England,Germany_England,GermanyvsEngland,DEENG,DE-ENG,DE_ENG,DEvsENG
1212,2021-07-06,1/2 FINAL,Italy,Spain,1-1,GOALS 1-0 (Chiesa 60’); 1-1 (Morata 80’),Italy,Spain,Roma,IT,...,IT_ES,ITvsES,SpainItaly,Spain-Italy,Spain_Italy,SpainvsItaly,ESIT,ES-IT,ES_IT,ESvsIT
1213,2021-07-07,1/2 FINAL,England,Denmark,2-1,GOALS 0-1 (Damsgaard 30’); 1-1 (Kjær [o.g.] 39...,England,Denmark,London,ENG,...,ENG_DK,ENGvsDK,DenmarkEngland,Denmark-England,Denmark_England,DenmarkvsEngland,DKENG,DK-ENG,DK_ENG,DKvsENG


In [41]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import os
import datetime

#Call function using keywords and date from matches. Load results in new dataframe.
tweets=pd.DataFrame()
for i in range(len(partidos["Date"])):
        keywords = [partidos['kw1'][i],partidos['kw2'][i],partidos['kw3'][i],partidos['kw4'][i],partidos['kw5'][i],
                    partidos['kw6'][i],partidos['kw7'][i],partidos['kw8'][i],partidos['kw9'][i],partidos['kw10'][i],
                    partidos['kw11'][i],partidos['kw12'][i],partidos['kw13'][i],partidos['kw14'][i],partidos['kw15'][i],
                    partidos['kw16'][i]]
        for j in range(len(keywords)):
            datos=scrape_tweets(keywords[j],partidos["Date"][i],partidos["capitales1"][i])
            tweets=tweets.append(datos,ignore_index=True)


    

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=Nigeria+since%3A2007-09-12+until%3A2007-09-13+&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retw

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=Nigeria+since%3A2007-09-12+until%3A2007-09-13+&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel failed, giving up.

In [25]:
a=pd.DataFrame()

a=a.append(datos)
a=a.append(datos,ignore_index=True)

In [6]:
datos

NameError: name 'datos' is not defined